## Part B:

You are given a dataset with 15 categories (dataset.zip). There are a total of 805 images.

1.For each class, treat images from 0001 to 0040 as train and the remaining as test.

2.Using pretrained resnet18, densenet121, vgg19 models on this dataset, train a 15-class classification model and report per-class classification accuracy in terms of precision and recall.

3.Finetune the vgg19 model on this dataset, train a 15-class classification model and report per-class classification accuracy in terms of precision and recall. 


## Project Introduction: 15-Class Image Classification Using Pretrained Models

In this project, we aim to develop and evaluate image classification models using a dataset consisting of 805 images 
spread across 15 categories.
Each category contains images labeled from 0001 to 0040 for training purposes,
while the remaining images are reserved for testing. 

## Objectives:
Here our objectives are two-fold:

**Pretrained Models Evaluation:**

* We will leverage the power of three state-of-the-art pretrained models: ResNet18, DenseNet121, and VGG19.
* For each model, we will fine-tune the final classification layers to adapt them to our 15-class dataset.
* We will train the models on the training set and evaluate their performance on the test set.
* We will report the per-class classification accuracy in terms of precision and recall.

**VGG19 Model Finetuning:**

* We will further fine-tune the VGG19 model specifically on our dataset.
* The model will be trained to classify the 15 categories and evaluated similarly.
* We will again report the per-class classification accuracy in terms of precision and recall.

In [1]:
#Importing the required libraries
import os                # To interact with the operating system for file and directory operations
import cv2               # OpenCV library for image processing tasks
import numpy as np       # NumPy library for numerical computations and array manipulations
import torch             # PyTorch library for building and training neural networks
import torch.nn as nn    # PyTorch module containing neural network layers and loss functions
import torch.optim as optim  # PyTorch module for optimization algorithms like SGD, Adam, etc.
from torchvision import transforms, datasets, models  # PyTorch module for image transformations, datasets, and pretrained models
from sklearn.metrics import classification_report     # Scikit-learn function to generate a classification report with precision, recall, etc.
from sklearn.preprocessing import LabelEncoder        # Scikit-learn class for encoding labels as numeric values
from sklearn.model_selection import train_test_split  # Scikit-learn function to split data into training and testing sets
from torch.utils.data import DataLoader, Dataset      # PyTorch classes for creating and loading custom datasets
from PIL import Image                                 # Pillow library for image manipulation and processing
import warnings                     # Python library to handle warnings, e.g., filter out unnecessary warnings during training
warnings.filterwarnings('ignore')

**Data Importing**

In [2]:
# Define the base directory and desired image size
base_dir = "D:/Term-5/Applications_of_AI/Group_Assignment/Q2_dataset"
desired_size = (224, 224)

# Initializing lists to store training and testing data and labels
image_data_train = []
image_data_test = []
labels_train = []
labels_test = []

# Loading images and labels
for category in os.listdir(base_dir):
    category_dir = os.path.join(base_dir, category)
    
    if os.path.isdir(category_dir):
        image_files = sorted(os.listdir(category_dir))
        for i, image_file in enumerate(image_files):
            image_path = os.path.join(category_dir, image_file)
            image = cv2.imread(image_path)
            
            if image is not None and image.size != 0:
                image = cv2.resize(image, desired_size)
                
                if i < 40:  # First 40 images for training
                    image_data_train.append(image)
                    labels_train.append(category)
                else:  # Rest for testing
                    image_data_test.append(image)
                    labels_test.append(category)



**Data Preprocessing**

In [3]:
# Convert lists to NumPy arrays
image_data_train = np.array(image_data_train)
image_data_test = np.array(image_data_test)
labels_train = np.array(labels_train)
labels_test = np.array(labels_test)

# Encode labels using LabelEncoder
le = LabelEncoder()
labels_train = le.fit_transform(labels_train)
labels_test = le.transform(labels_test)

## EDA

In [4]:
def count_images_in_dataset(base_dir):
    total_images = 0
    category_image_counts = {}

    for category in os.listdir(base_dir):
        category_dir = os.path.join(base_dir, category)
        
        if os.path.isdir(category_dir):
            image_files = [f for f in os.listdir(category_dir) if os.path.isfile(os.path.join(category_dir, f))]
            num_images = len(image_files)
            category_image_counts[category] = num_images
            total_images += num_images

    return total_images, category_image_counts

total_images, category_image_counts = count_images_in_dataset(base_dir)

print(f"Total number of images: {total_images}")
for category, count in category_image_counts.items():
    print(f"Category '{category}' has {count} images")


Total number of images: 805
Category 'accordion' has 55 images
Category 'bass' has 54 images
Category 'camera' has 50 images
Category 'crocodile' has 50 images
Category 'crocodile_head' has 51 images
Category 'cup' has 57 images
Category 'dollar_bill' has 52 images
Category 'emu' has 53 images
Category 'gramophone' has 51 images
Category 'hedgehog' has 54 images
Category 'nautilus' has 55 images
Category 'pizza' has 53 images
Category 'pyramid' has 57 images
Category 'sea_horse' has 57 images
Category 'windsor_chair' has 56 images


In [5]:
#Checking the count in training & test datasets
def count_images_in_dataset(base_dir):
    train_image_counts = {}
    test_image_counts = {}
    total_train_images = 0
    total_test_images = 0

    for category in os.listdir(base_dir):
        category_dir = os.path.join(base_dir, category)
        
        if os.path.isdir(category_dir):
            image_files = sorted([f for f in os.listdir(category_dir) if os.path.isfile(os.path.join(category_dir, f))])
            num_train_images = len(image_files[:40])  # First 40 images for training
            num_test_images = len(image_files[40:])  # Remaining images for testing

            train_image_counts[category] = num_train_images
            test_image_counts[category] = num_test_images

            total_train_images += num_train_images
            total_test_images += num_test_images

    return total_train_images, total_test_images, train_image_counts, test_image_counts

total_train_images, total_test_images, train_image_counts, test_image_counts = count_images_in_dataset(base_dir)

print(f"Total number of training images: {total_train_images}")
print(f"Total number of testing images: {total_test_images}\n")

print("Training set image counts per category:")
for category, count in train_image_counts.items():
    print(f"Category '{category}' has {count} images")

print("\nTesting set image counts per category:")
for category, count in test_image_counts.items():
    print(f"Category '{category}' has {count} images")


Total number of training images: 600
Total number of testing images: 205

Training set image counts per category:
Category 'accordion' has 40 images
Category 'bass' has 40 images
Category 'camera' has 40 images
Category 'crocodile' has 40 images
Category 'crocodile_head' has 40 images
Category 'cup' has 40 images
Category 'dollar_bill' has 40 images
Category 'emu' has 40 images
Category 'gramophone' has 40 images
Category 'hedgehog' has 40 images
Category 'nautilus' has 40 images
Category 'pizza' has 40 images
Category 'pyramid' has 40 images
Category 'sea_horse' has 40 images
Category 'windsor_chair' has 40 images

Testing set image counts per category:
Category 'accordion' has 15 images
Category 'bass' has 14 images
Category 'camera' has 10 images
Category 'crocodile' has 10 images
Category 'crocodile_head' has 11 images
Category 'cup' has 17 images
Category 'dollar_bill' has 12 images
Category 'emu' has 13 images
Category 'gramophone' has 11 images
Category 'hedgehog' has 14 images


In [6]:
# Custom Dataset class
class CustomDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        image = Image.fromarray(image)

        if self.transform:
            image = self.transform(image)

        return image, label

# Data Transformations (same for all models)
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Datasets and Data Loaders
train_dataset = CustomDataset(image_data_train, labels_train, transform=data_transforms['train'])
test_dataset = CustomDataset(image_data_test, labels_test, transform=data_transforms['test'])

dataloaders = {
    'train': DataLoader(train_dataset, batch_size=32, shuffle=True),
    'test': DataLoader(test_dataset, batch_size=32)
}



In [7]:
# Model Training and Evaluation Function
def train_and_evaluate_model(model_name, num_classes=15):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = getattr(models, model_name)(pretrained=True)

    if model_name.startswith("densenet"):
        num_ftrs = model.classifier.in_features
        model.classifier = nn.Linear(num_ftrs, num_classes)
    else:
        num_ftrs = model.fc.in_features
        model.fc = nn.Linear(num_ftrs, num_classes)

    model = model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

    # Training loop (simplified)
    for epoch in range(15):
        model.train()
        for inputs, labels in dataloaders['train']:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

    # Evaluation (using sklearn's classification_report)
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in dataloaders['test']:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    class_names = le.classes_
    report = classification_report(all_labels, all_preds, target_names=class_names, output_dict=False)
    print(f"\nClassification Report for {model_name}:\n{report}")

# Train and Evaluate Each Model
for model_name in ['resnet18', 'densenet121']:
    train_and_evaluate_model(model_name)



Classification Report for resnet18:
                precision    recall  f1-score   support

     accordion       1.00      1.00      1.00        15
          bass       0.93      1.00      0.97        14
        camera       1.00      1.00      1.00        10
     crocodile       0.82      0.90      0.86        10
crocodile_head       0.90      0.82      0.86        11
           cup       1.00      1.00      1.00        17
   dollar_bill       1.00      1.00      1.00        12
           emu       1.00      1.00      1.00        13
    gramophone       1.00      0.91      0.95        11
      hedgehog       0.93      0.93      0.93        14
      nautilus       0.94      1.00      0.97        15
         pizza       1.00      1.00      1.00        13
       pyramid       1.00      1.00      1.00        17
     sea_horse       1.00      0.94      0.97        17
 windsor_chair       1.00      1.00      1.00        16

      accuracy                           0.97       205
     macr